In [1]:
import pandas as pd
import numpy as np
from symbol import Symbol
from index import Index
from portifolio import Portifolio
from datetime import datetime
import os
import matplotlib.pyplot as plt
from bqplot import pyplot as bplt
import seaborn as sns
from dateutil.relativedelta import relativedelta

sns.set(context='notebook', style="dark", palette="muted", color_codes=True)

%matplotlib inline

In [2]:
def plot_close(hist_data, symbol_list):
    grouped_data = hist_data.groupby('symbol')
    try:
        bplt.close(1)
    except:
        pass
    bplt.figure(1, title='Line Chart')
    bplt.clear()
    for symbol in symbol_list:
        adjclose = grouped_data.get_group(symbol).set_index('date').adj_close
        bplt.plot(x=adjclose.index, y=adjclose, marker_str=symbol)
        bplt.legend()
    
    bplt.show()

def plot_returns(returns_data, symbol_list):
    try:
        bplt.close(1)
    except:
        pass
    bplt.figure(1, title='Line Chart')
    bplt.clear()
    for symbol in symbol_list:
        bplt.plot(x=returns_data.index, y=returns_data[[symbol]])
        bplt.legend()
    
    bplt.show()

In [3]:
s= Symbol()
i = Index()
p = Portifolio(start=2016)

In [5]:
ret = s.get_symbol_data(null_count=5)
ret = ret.fillna(0, limit=5)
close = s.get_symbol_data(data='close', null_count=5)
iret = i.get_index_returns()
iret = iret.fillna(0, limit=5)
common_index = np.intersect1d(ret.index, iret.index)
ret = ret.loc[common_index]
iret = iret.loc[common_index]
close= close.loc[common_index]
infy_ret = ret.infy
indret = iret.nifty_50
adj_close = ret.apply(p.prices)
len(ret), len(ret.columns), len(iret)

(5240, 40, 5240)

In [8]:
# def min_52w(returns):
#     if isinstance(returns, pd.Series):
#         returns = pd.DataFrame(returns)
#     else:
#         min_52w = returns.apply(min_52w)
#         return min_52w
#     returns['max'] = np.nan
#     returns['min'] = np.nan
#     for index in returns.index:
#         returns.loc[index, 'max'] = returns.loc[index - relativedelta(years=1):index, index.columns[0]].max()
#         returns.loc[indi, 'min'] = returns.loc[index - relativedelta(years=1):index, index.columns[0]].min()
#     inf = inf[req]
#     return inf
# adj_close[['infy', 'wipro']].apply(min_max_52w, req='max')
from pandas.tseries.offsets import *
close.rolling(248).min().infy.max()

5891.3500000000004

In [ ]:
conditions = [
    'holding_period_return > 0',
    'excess_returns_rf > 0',
    'excess_returns_bench > 0',
    'num_returns > 500',
    'upside_potential_ratio > 1',
    'pos_pctile > 0.50',
# #     'min_returns > -0.15',
#     'std_dev < 0.03',
#     'sortino_ratio > 1'
]
str_cond = ' & '.join(conditions)
if str_cond == '':
    ind = desc_ret.index
else:
    ind = desc_ret.query(str_cond).index
desc_ret.loc[ind].sort_values('excess_returns_bench', ascending=False)
# desc_ret.loc['infy']
(desc_ret.excess_returns_bench/desc_ret.num_returns).sort_values(ascending=False)
pq = desc_ret.query('num_returns > 1000 & excess_returns_bench > 0').eval('excess_returns_bench/num_returns').sort_values(ascending=False)
(pq/desc_ret.std_dev).dropna().sort_values(ascending=False)

In [ ]:
conditions = [
    'holding_period_return > 0.1',
    'upside_potential_ratio > 1',
    'pos_pctile > 0.5',
    'min_returns > -0.15',
    'std_dev < 0.03',
    'sortino_ratio > 1'
]
str_cond = ' & '.join(conditions)
ind = desc_ret16.query(str_cond).index
desc_ret16.loc[ind]

In [ ]:
desc_ret16.loc[ind]

In [ ]:
conditions = [
    'holding_period_return > 5 * 0.1',
    'upside_potential_ratio > 1',
    'pos_pctile > 0.5',
    'min_returns > -0.15',
    'std_dev < 0.03',
    'sortino_ratio > 1',
    'beta > 0.8'
]
str_cond = ' & '.join(conditions)
ind = desc_ret15.query(str_cond).index
desc_ret15.loc[ind]

In [ ]:
desc_ret16[desc_ret16.index.isin(ind)]

In [ ]:
vol = s.get_symbol_data(data='volatility', start='2015',null_count=5, volume=1000)
vol.mean().sort_values().head()

In [ ]:
np.sqrt(np.floor_divide(len(returns), interval)+1)

In [ ]:
iport = s.returns[['INFY', 'BPCL']]['2000':].fillna(0)/100
start_value = 10000
weights = np.ones(len(iport.columns))/len(iport.columns)
myport = pd.DataFrame(0, index=iport.index.copy(), columns = iport.columns.copy())
myport.ix[0, :] = start_value/(len(iport.columns))
myport = myport.ix[0] * np.exp(iport.cumsum())
myport['value'] = myport.sum(axis=1)
myport.tail()
# # myport = myport/myport.ix[0]

In [ ]:
def calc_portfolio_var(returns, weights=None):
    if weights is None: 
        weights = np.ones(returns.columns.size) / \
        returns.columns.size
    sigma = np.cov(returns.T,ddof=0)
    var = (weights * sigma * weights.T).sum()
    return var

In [ ]:
def sharpe_ratio(returns, weights = None, risk_free_rate = 0.045):
    n = returns.columns.size
    if weights is None: weights = np.ones(n)/n
    # get the portfolio variance
    var = calc_portfolio_var(returns, weights)
    # and the means of the stocks in the portfolio
    means = returns.mean()
    # and return the sharpe ratio
    risk_free_rate = np.log(1+risk_free_rate)/365
    return (means.dot(weights) - risk_free_rate)/np.sqrt(var)
sharpe_ratio(iport)
# iport.mean()

In [ ]:
def y_f(x): return 2+x**2

import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats
scopt.fmin(y_f, 1000)

In [ ]:
def negative_sharpe_ratio_n_minus_1_stock(weights, 
                                          returns, 
                                          risk_free_rate):
    """
    Given n-1 weights, return a negative sharpe ratio
    """
    weights2 = sp.append(weights, 1-np.sum(weights))
    return -sharpe_ratio(returns, weights2, risk_free_rate)
def optimize_portfolio(returns, risk_free_rate):
    """ 
    Performs the optimization
    """
    # start with equal weights
    w0 = np.ones(returns.columns.size-1, 
                 dtype=float) * 1.0 / returns.columns.size
    # minimize the negative sharpe value
    w1 = scopt.fmin(negative_sharpe_ratio_n_minus_1_stock, 
                    w0, args=(returns, risk_free_rate))
    # build final set of weights
    final_w = sp.append(w1, 1 - np.sum(w1))
    # and calculate the final, optimized, sharpe ratio
    final_sharpe = sharpe_ratio(returns, final_w, risk_free_rate)
    return (final_w, final_sharpe)
optimize_portfolio(iport, 0.045)

In [ ]:
def objfun(W, R, target_ret):
    stock_mean = np.mean(R,axis=0)
    port_mean = np.dot(W,stock_mean) # portfolio mean
    cov=np.cov(R.T) # var-cov matrix
    port_var = np.dot(np.dot(W,cov),W.T) # portfolio variance
    penalty = 2000*abs(port_mean-target_ret)# penalty 4 deviation
    return np.sqrt(port_var) + penalty # objective function
def calc_efficient_frontier(returns):
    result_means = []
    result_stds = []
    result_weights = []
    
    means = returns.mean()
    min_mean, max_mean = means.min(), means.max()
    
    nstocks = returns.columns.size
    
    for r in np.linspace(min_mean, max_mean, 100):
        weights = np.ones(nstocks)/nstocks
        bounds = [(0,1) for i in np.arange(nstocks)]
        constraints = ({'type': 'eq', 
                        'fun': lambda W: np.sum(W) - 1})
        results = scopt.minimize(objfun, weights, (returns, r), 
                                 method='SLSQP', 
                                 constraints = constraints,
                                 bounds = bounds)
        if not results.success: # handle error
            raise Exception(result.message)
        result_means.append(np.round(r,4)) # 4 decimal places
        std_=np.round(np.std(np.sum(returns*results.x,axis=1)),6)
        result_stds.append(std_)
        
        result_weights.append(np.round(results.x, 5))
    return {'Means': result_means, 
            'Stds': result_stds, 
            'Weights': result_weights}
frontier_data = calc_efficient_frontier(iport)
# first five risk levels
frontier_data['Stds'][:5]
# first five mean returns
frontier_data['Means'][:5]
# first five sets of optimal weights
frontier_data['Weights'][:5]

In [ ]:
def plot_efficient_frontier(frontier_data):
    plt.figure(figsize=(12,8))
    plt.title('Efficient Frontier')
    plt.xlabel('Standard Deviation of the porfolio (Risk))')
    plt.ylabel('Return of the portfolio')
    plt.plot(frontier_data['Stds'], frontier_data['Means'], '--'); 
    
plot_efficient_frontier(frontier_data)

In [ ]:
z = spstats.norm.ppf(0.95)
VaR = 10000 * (z * iport.INFY.std())
VaR

In [ ]:
import matplotlib.mlab as mlab
# draw a 99% one-tail confidence interval
x = np.linspace(-4,4,101)
y = np.exp(-x**2/2) / np.sqrt(2*np.pi)
x2 = np.linspace(-4,-2.33,101)
y2 = np.exp(-x2**2/2) / np.sqrt(2*np.pi)
f = plt.figure(figsize=(12,8))
plt.plot(x,y*100, linewidth=2)
xf, yf = mlab.poly_between(x2, 0*x2, y2*100)
plt.fill(xf, yf, facecolor='g', alpha=0.5)
plt.gca().set_xlabel('z-score')
plt.gca().set_ylabel('Frequency %')
plt.title("VaR based on the standard normal distribution")
bbox_props = dict(boxstyle="rarrow,pad=0.3", fc="w", ec="b", lw=2)
t = f.text(0.25, 0.35, "99% VaR confidence level", ha="center", va="center", 
           rotation=270,
            size=15,
            bbox=bbox_props)
plt.savefig('5104OS_09_21.png', bbox_inches='tight', dpi=300)

In [ ]:
ret[['GALLISPAT', 'INFY']].plot()

In [ ]:
get_hist_data(hist_data, 'INFY')['2001-04']

In [ ]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

In [ ]:
ret.dropna(axis=1)

In [ ]:
symbol_list['INFY']